In [18]:
param_dir = 'data_20240206'
param_make = 'CROSLEY'

In [19]:
import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

In [20]:
input_ = pd.read_excel('./' + param_dir + '/' + param_make + '/5.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
input_['Category'].tolist()

总数量：17


['Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Drivetrain',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer',
 'Wiper & Washer']

In [21]:
crawler_status = 'error'

print(crawler_status)

error


In [22]:
import requests
import json

In [23]:
import sys
sys.path.append('../../../00.module')
import UA
import Proxy

i = 0

jsn = '{"jsn":{"tab":"catalog","make":"' + input_.loc[i, 'Make'] + '","year":"' + input_.loc[i, 'Year'] + '","model":"' + input_.loc[i, 'Model'] + '","engine":"' + input_.loc[i, 'Engine'] + '","carcode":"' + input_.loc[i, 'Car_Code'] + '","groupname":"' + input_.loc[i, 'Category'] + '","nodetype":"groupname"}}'

payload = {'func': 'navnode_fetch',
           'payload': jsn,
           'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=payload,
                                         proxies=Proxy.get_Proxy_Requests()).text

resp = json.loads(resp)

resp

{'html_fill_sections': {'navchildren[]': ''},
 'jsn': {'tab': 'catalog',
  'make': 'CROSLEY',
  'year': '1952',
  'model': 'PANEL DELIVERY',
  'engine': '724cc 44cid L4',
  'carcode': '1487439',
  'groupname': 'Wiper & Washer',
  'nodetype': 'groupname',
  'idepth': 0},
 'collected_javascript': '/**UQ:012FC849**/\nwindow.top.parent.window["_nck"] = "8LVLUxTaILBT+tLxkg8+3Ag7\\/0bbOhvn7P3rkFbL80\\/oQB6RvzNfa5dcHvATqXAMxw5whzaDEdxk+27zeij67BjKru6WqTdNqG6tiEsD1EZhbTv0XForkbYp+nLaIKxDXfb4DjpqS2ZyAqHw8OmZkA2eIp8YUxluzTN7FEXkPw4=";\n'}

In [24]:
from bs4 import BeautifulSoup

In [25]:
html = resp['html_fill_sections']['navchildren[]']

soup = BeautifulSoup(html, 'lxml')

with open('./type.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

In [26]:
from lxml import etree

In [27]:
html = etree.HTML(str(soup))

list_type = [type_.strip() for type_ in html.xpath('//td[@class="nlabel"]/a/text()')]

list_type

AttributeError: 'NoneType' object has no attribute 'xpath'

In [ ]:
list_url = ['https://www.rockauto.com'+url.strip() for url in html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')]

list_url

In [ ]:
crawler_status = 'ok'

print(crawler_status)

In [ ]:
if crawler_status == 'ok':
    df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                            'No.': int(input_.loc[i, 'No.']),
                            'Make': input_.loc[i, 'Make'],
                            'Year': input_.loc[i, 'Year'],
                            'Model': input_.loc[i, 'Model'],
                            'Engine': input_.loc[i, 'Engine'],
                            'Car_Code': input_.loc[i, 'Car_Code'],
                            'Category': input_.loc[i, 'Category'],
                            'Type': list_type,
                            'Type_Code': [url.split(',')[-1] for url in list_url],
                            'Url': list_url})
    
    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
else:
    df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                             'No.': int(input_.loc[i, 'No.']),
                             'Make': input_.loc[i, 'Make'],
                             'Year': input_.loc[i, 'Year'],
                             'Model': input_.loc[i, 'Model'],
                             'Engine': input_.loc[i, 'Engine'],
                             'Car_Code': input_.loc[i, 'Car_Code'],
                             'Category': input_.loc[i, 'Category'],
                             'Url': input_.loc[i, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
print(len(output_correct), len(output_error))
output_correct